# EuroPython program grid

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [2]:
import json
import datetime as dt
from random import choice, randrange, shuffle
from copy import deepcopy

from collections import OrderedDict, defaultdict
from itertools import product
from functools import partial
from operator import itemgetter

from eptools.dict_query import build_query, run_query

from IPython.display import display, HTML
show = lambda s: display(HTML(s))

## Load the data

In [3]:
talk_sessions = json.load(open('accepted_talks.json'))

In [4]:
list(talk_sessions.keys())

['talk',
 'Opening session',
 'Closing session',
 'training',
 'Lightning talk',
 'poster',
 'EPS session']

## Clean up the data

Here I pick from `talk_sessions` only the talks with the type that I need for scheduling.

I also remove from all these talks the fields that I don't need for scheduling, to maintain the `prints` clean and short enough.

In [5]:
#all talks
all_talks = []
for s in talk_sessions.values():
    all_talks.extend(list(s.values()))

#the talks worth for scheduling
grid_talks = []
sessions = talk_sessions.copy()
general_grid_sessions = ['talk', 'training']
for session_name in general_grid_sessions:
    grid_talks.extend(sessions[session_name].values())

fields2pop = ['abstract_extra', 
              'abstract_long',
              'abstract_short',
              'twitters',
              'emails',
              'status',
              'url',
              'companies',
              'have_tickets',
             ]
for talk in grid_talks:
    for f in fields2pop:
        talk.pop(f)

## Declare the week schedule

Declare certain structures to be able to declare and define the conference schedule.

The information here will be used in the `dict_query` submodule to filter the talks.

In [58]:
tags_field = 'tag_categories'

weekday_names = {0: 'Monday, July 18th', 
                 1: 'Tuesday, July 19th',
                 2: 'Wednesday, July 20th',
                 3: 'Thursday, July 21st',
                 4: 'Friday, July 22nd'
                }

room_names = {0: 'A1',
              1: 'A3',
              2: 'A2',
              3: 'Ba1',
              4: 'Ba2',
              5: 'E' ,
              6: 'A4',
             } 

# this is not being used yet
durations = {'announcements': 15,
             'keynote': 45,
             'lts': 60,
             'lunch': 60,
             'am_coffee': 30,
             'pm_coffee': 30,
            }

# track schedule types, by talk conditions
track_schedule1 = [(('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', (45, 60)), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', 45)), ),
                   #(('admin_type', 'Lightning talk'), ),
                  ]

track_schedule2 = [(('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', (45, 30)), ),
                   (('duration', (60, 45)), ),
                   (('duration', (60, 45)), ),
                   (('duration', 45), ),
                  ]

track_schedule3 = [(('duration', 45), ), 
                   (('duration', 45), ),
                   (('duration', (45, 30)), ),
                   (('duration', (45, 60)), ),
                   (('duration', (45, 30), ),
                   (('duration', (60, 30)), ),
                   (('duration', (45, 30)), ),
                  ]

#tutorials
track_schedule4 = [(('type', 'has Training'), ), 
                   (('type', 'has Training'), ), ]


# these are for reference, but not being taken into account (yet)
frstday_schedule1 = [(('admin_type', 'Opening session')), 
                     (('admin_type', 'Keynote')),
                    ] + track_schedule1

lastday_schedule1 = track_schedule1 + [(('admin_type', 'Closing session')),]

# I removed time from here.
#daily_timegrid = lambda schedule: OrderedDict([(datetime.time(*slot[0]), slot[1]) for slot in schedule])

room1_schedule = track_schedule1 # A1, the google room
room2_schedule = track_schedule2 # A3, pythonanywhere room
room3_schedule = track_schedule3 # A2
room4_schedule = track_schedule3 # Barria1
room5_schedule = track_schedule3 # Barria2

room6_schedule = track_schedule4 # Room E
room7_schedule = track_schedule4 # Room A4

daily_schedule   = OrderedDict([(0, room1_schedule), 
                                (1, room2_schedule),
                                (2, room3_schedule),
                                (3, room4_schedule),
                                (4, room5_schedule),
                                (5, room6_schedule),
                                (6, room7_schedule)])

# week conditions
default_condition = (('language', 'English'), ('type', 'has Talk'),)

# [day][room] -> talk conditions
dayroom_conditions = {0: {},
                      1: {4: (('language', 'Spanish'), ), },
                      2: {3: (('language', 'Basque' ), ), },
                      3: {},    
                      4: {},    
                     }

SyntaxError: invalid syntax (<ipython-input-58-9a83d7d2367b>, line 35)

## Build the schedule conditions table

In [59]:
# the whole schedule conditions table

def join_conds(condset1, condset2):
    d = dict(condset1)
    if condset2:
        d.update(dict(condset2))
    return tuple(d.items())


week_conditions = defaultdict(dict)
for day, room in product(weekday_names, room_names):
    track_schedule = daily_schedule[room]
    dayroom_conds  = dayroom_conditions[day].get(room, default_condition)
    week_conditions[day][room] = []
    for slot_conds in track_schedule:
        week_conditions[day][room].append(join_conds(dayroom_conds, slot_conds))

In [60]:
week_conditions[0][5]

[(('language', 'English'), ('type', 'has Training')),
 (('language', 'English'), ('type', 'has Training'))]

## Group tags and count talks-per-tag

In [61]:
tags2pop = ['>>> Suggested Track', 'Python', '']
tags = defaultdict(int)

for talk in all_talks:
    for t in talk[tags_field]:
        if t in tags2pop:
            continue            
        tags[t] += 1
tags_sorted = sorted(tags.items(), key=itemgetter(1), reverse=True)
tags_sorted

[('Programming', 94),
 ('Data Science', 88),
 ('DevOps', 64),
 ('Best Practice and Use Cases', 48),
 ('Educational', 29),
 ('Web', 27),
 ('Community', 25),
 ('Testing', 24),
 ('Open Source', 17),
 ('Databases', 15),
 ('Everything Else', 14),
 ('Hardware', 13),
 ('Security', 12),
 ('Other Programming Languages', 11),
 ('Case Study', 11),
 ('Sciences', 10),
 ('Application Frameworks', 8),
 ('Development Methods', 2),
 ('Operating Systems', 2),
 ('Business', 1)]

## Filtering functions

Here I declare the functions used to filter talks using the `dict_query`-type queries defined above.

In [99]:
def pick_talk(talks, conditions, trialno=1):
    if not talks:
        raise IndexError('The list of talks is empty!')

    query = build_query(conditions)

    for tidx, talk in enumerate(talks):
        if run_query(talk, query):
            return talks.pop(tidx)
    
    # if no talk fills the query requirements
    if trialno == 1:
        nuconds = dict(conditions)
        del nuconds[tags_field]
        nuconds = tuple(nuconds.items())
        print('2ND TRY: Looking only for {}.'.format(nuconds))
        return pick_talk(talks, nuconds, trialno=2)
    if trialno == 2:
        oldconds = dict(conditions)
        nuconds  = {}
        if 'duration' in oldconds:
            nuconds['duration'] = oldconds['duration']
        if 'type' in oldconds:
            nuconds['type'] = oldconds['type']
        nuconds = tuple(nuconds.items())
        print('3RD TRY: Looking only for {}.'.format(nuconds))
        return pick_talk(talks, nuconds, trialno=3)
    else:
        print('FAILED looking for {}.'.format(conditions))
        return {}

In [100]:
# collections splitting utilities

import random

def chunks(l, n):
    """Yield successive `n`-sized chunks from `l`."""
    for i in range(0, len(l), n):
        yield l[i:i+n]
        

def split(xs, n):
    """ Yield `n` chunks of the sequence `xs`."""
    ys = list(xs)
    random.shuffle(ys)
    size = len(ys) // n
    leftovers = ys[size*n:]
    for c in range(n):
        if leftovers:
            extra = [ leftovers.pop() ] 
        else:
            extra = []
        yield ys[c*size:(c+1)*size] + extra

## Distribute the talks along the schedule

In [101]:
from eptools.dict_query import or_condition

talks = grid_talks.copy()
shuffle(talks)


def condition_set(slot_conditions, default_conditions, topic_conditions):
    conds = join_conds(default_conditions, slot_conditions)
    if 'admin_type' not in dict(conds):
        conds = join_conds(conds, topic_conditions)
    return conds

# random pick talks
week_slots = defaultdict(dict)
for day in weekday_names:
    shuffle(tags_sorted)
    tags_chunks  = list(split([t[0] for t in tags_sorted], len(room_names)))
    rooms_topics = {room: or_condition(tags_field, 'has', tags) 
                   for room, tags in zip(room_names.keys(), tags_chunks)}  
    
    for room in room_names:
        slots_conds = week_conditions[day][room]
        room_topics = rooms_topics[room]
        week_slots[day][room] = []
        #print(len(talks))
        for slot_cond in slots_conds:
            conds = condition_set(slot_cond, default_condition, room_topics)
            try:
                week_slots[day][room].append(pick_talk(talks, conds))
            except IndexError:
                print('No talks left for {}.'.format(conditions))
            except:
                raise

2ND TRY: Looking only for (('duration', 45), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', 45), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', (45, 60)), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', 45), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', (60, 30)), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', (45, 30)), ('language', 'Spanish'), ('type', 'has Talk')).
2ND TRY: Looking only for (('language', 'English'), ('type', 'has Training')).
2ND TRY: Looking only for (('duration', 45), ('language', 'English'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', 45), ('language', 'English'), ('type', 'has Talk')).
2ND TRY: Looking only for (('duration', 45), ('language', 'Basque'), ('type', 'has Talk')).
3RD TRY: Looking only for (('duration', 45), ('type', 'has Talk')).

### Remaining talks

Print the remaining talks that have been left out of the schedule (by accident?).

In [102]:
q = build_query((('type', 'has Training'),))
run_query(talks[0], q)

False

In [103]:
if talks:
    show('<h1>Not scheduled talks</h1>')

    for talk in talks:
        print(talk)

{'sub_community': '', 'admin_type': '', 'tags': ['Web', 'General'], 'sub_title': '', 'language': 'Basque', 'track_title': '', 'level': 'Beginner', 'tag_categories': ['', ''], 'title': 'Datu bistaratze soluzioen garapena Smartcity proiektuetan', 'id': 693, 'timerange': '', 'duration': 30, 'type': 'Talk (30 mins)', 'speakers': 'Iker Martinez de Agirre Mendia'}
{'sub_community': '', 'admin_type': '', 'tags': ['Python general', 'Packaging', 'Cross-Platform-Development'], 'sub_title': 'A better solution to the packing problem!?', 'language': 'English', 'track_title': '', 'level': 'Beginner', 'tag_categories': ['Python', 'Python', 'Python'], 'title': 'Conda - Easier Installs and Simpler Builds', 'id': 597, 'timerange': '', 'duration': 30, 'type': 'Talk (30 mins)', 'speakers': 'Mike Müller'}
{'sub_community': '', 'admin_type': '', 'tags': ['python'], 'sub_title': '', 'language': 'English', 'track_title': '', 'level': 'Beginner', 'tag_categories': [''], 'title': 'Sponsored Talk', 'id': 701, 't

## Print the schedule

Declare functions needed to orederly access the talks in the filled schedule and print the tables nicely in this notebook.

In [104]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


def tabulate(time_list, header=''):
    table = ListTable()
    table.append(header)
    for slot in time_list:
        table.append([slot] + time_list[slot])
    return table


def get_room_schedule(weekly_schedule, room_name, field='title'):
    slots = list(daily_schedule[room_name].keys())
    daily_slots = []
    for slot in slots:
        talks = [weekly_schedule[d][room_name][slot].get(field, '-') for d in range(n_days)]
        daily_slots.append((slot, talks))
    room_schedule = OrderedDict(daily_slots)
    return room_schedule


from itertools import zip_longest
def get_day_schedule(weekly_schedule, day_num, field='title'):
    day_schedule = weekly_schedule[day_num]
    nslots = max([len(slots) for room, slots in day_schedule.items()])
    room_slots = []
    for room, talk_slots in day_schedule.items():
        room_talks = [talk.get(field, '-') for slot, talk in enumerate(talk_slots)]
        room_slots.append(room_talks)
    schedule = OrderedDict(list(enumerate(list(map(list, zip_longest(*room_slots))))))
    return schedule

## Schedule

In [105]:
sched_field = 'title'

for day, _ in enumerate(weekday_names):
    show('<h3>{}</h3>'.format(weekday_names[day]))
    show(tabulate(get_day_schedule(week_slots, day), 
                  header=['Slot'] + list(room_names.values()))._repr_html_())

Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Implementing Parallel Programming Design Patterns using EFL for Python,Jupyter for everything else,SQLAlchemy as the backbone of a Data Science company,Handling GPS Data with Python,Build your first OpenStack application with OpenStack PythonSDK,The Stupid Python Workshop,Property-based testing with Hypothesis
1,Against the silos: usable encrypted email & the quest for privacy-aware services,Get Instrumented!,TDD of Python microservices,CloudABI: Capability based security on Linux/UNIX,How to improve your diet and save money with Python,"Blender: much visual, very 3d, many python.",OpenStack Cloud Native Deployment for Application Developers
2,The Report Of Twisted’s Death,AFP: secure cloud authentication for machines and humans.,Real virtual environments without virtualenv,Developing a real-time automated trading platform with Python,Is that spam in my ham?,None,None
3,Scaling Microservices with Crossbar.io,Introduction to aiohttp,Operating on Encrypted Data with ZeroDB,How OpenStack makes Python better (and vice-versa),Kung Fu at Dawn with Itertools,None,None
4,Things I wish I knew before starting using Python for Data Processing,Ingesting 35 million hotel images with python in the cloud.,What is the best full text search engine for Python?,Brainwaves for hackers 3,The Joy of Simulation: for Fun and Profit,None,None
5,"Automate, contribute, repeat.",Game Theory to the Rescue When Hard Decisions Are to Be Made,Building a mBaaS framework using Django,Making robots walk with Python,Pure Data and a Clean Architecture,None,None
6,Implementing a Sound Identifier in Python,System Testing with pytest and docker-py,AWS lambda & Python,Clean code in Python,Testing the untestable: a beginner’s guide to mock objects,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Python in Astronomy,Building your own AI,"I Hate You, NLP... ;)",Do I need to switch to Go(lang) ?,Hacking ético con herramientas Python,Guide to make a real contribution to an open source project for novice,"So, what's all the fuss about Docker?"
1,High Performance Networking in Python,What's the point of Object Orientation?,Analyzing Data with Python & Docker,Towards More Secure Emailing,Kung Fu al amanecer con itertools,"pytest - simple, rapid and fun testing with Python","Faster Python Programs - Measure, don't Guess"
2,Nipy on functional brain MRI,Python Descriptors for Better Data Structures,Wrestling Python into LLVM Intermediate Representation,Python and Async programming,Implementación de un Identificador de Sonido en Python,None,None
3,Data Formats for Data Science,Profiling the unprofilable,An Introduction to Deep Learning,Ethical hacking with Python tools,APIs and Microservices With Go,None,None
4,"Beyond scraping, getting data from dynamic, heavily javascript driven, websites",Effectively test your webapp with Python and Selenium,Effective Code Review,Moving away from NodeJS to a pure python solution for assets,Un vector por tu palabra,None,None
5,Fast Async Code with Cython and AsyncIO,Pygame Zero,NetworkX Visualization Powered by Bokeh,Behind Closed Doors: Managing Passwords in a Dangerous World,Entendiendo Unicode,None,None
6,Optimize Thyself,Keeping the Lights on with Python,Predicting Oscar Winners & Box Office Hits with Scikit Learn,Monkey-patching: a magic trick or a powerful tool?,Pytest desde las trincheras,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Deep Learning with Python & TensorFlow,Using Service Discovery to build dynamic python applications,Performant Python,FAT Python: a new static optimizer for Python 3.6,Efficient Django,Manage your Python packages professionally with devpi,Efficient Python for High-Performance Parallel Computing
1,A Gentle Introduction to Neural Networks (with Python),What Python can learn from Haskell packaging,Asynchronous network requests in a web application,Metaclasses for fun and profit: Making a declarative GUI implementation,-,Test-driven code search and the art of writing implementation-agnostic tests,"(Machine-)Learning Chinese, with Python!"
2,"OMG, Bokeh is better than ever!",Protect your users with Circuit Breakers,"Iteration, iteration, iteration",Buildout Django eta Fabric. Kasu praktikoa euskarazko tokiko hedabideetan,import community,None,None
3,Unveiling the Universe with python,"Infrastructure as Code: ""pip install"" your environment",Raspberry Pi GPIO Zero,Modern OpenGL with Python,A deep dive into the Pymongo MongoDB driver,None,None
4,Python in Gravitational Waves Research Communities,MicroPython on the BBC micro:bit,Designing a Pythonic Interface,Building Service interfaces with OpenAPI / Swagger,-,None,None
5,It's not magic: descriptors exposed,"Grocker, a Python build chain for Docker",Writing faster Python,EITB Nahieran: askatu bideoak API honen bidez,The Journey from Python Developer to Python Company Owner,None,None
6,Music transcription with Python,MiniBrew: Brewing beer with Python,Building beautiful RESTful APIs using Flask,"Endor, ipuinak kontatzen zituen Nao robota.","Server for IoT devices and Mobile devices using Wifi Network,",None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,-,-,-,-,-,Python for System Administrators,uWSGI: the full stack application server
1,-,-,-,-,-,Introduction to Deep Learning for Natural Language Processing,NumPy with Cython
2,-,Writing Redis in Python with asyncio,Create secure production environment using Docker,Machine Learning for dummies with Python,The right way to write documentation,None,None
3,-,-,-,-,-,None,None
4,-,So you think your Python startup is worth $10 million...,-,-,-,None,None
5,-,Building a reasonably popular web application for the first time.,Dynamic Class Generation in Python,Machine Learning: Power of Ensembles,Writing Python Native Extensions in Rust,None,None
6,Peeking into Python’s C API,"Python, Data & Rock'n'Roll",Managing Mocks,Simplifying Computer Art in Python,CFFI: calling C from Python,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,-,-,-,-,-,Introduction to Data Wrangling,IPython in Depth
1,-,-,-,-,-,Productive Coding with PyCharm,Present-day Async Web development training: from Twisted to Tornado and AsyncIO
2,-,How to use Metaclasses to improve your Software Design,Build your Microservices with ZeroMQ,RESTful API - Best Practises.,Learn Python The Fun Way,None,None
3,-,-,-,-,-,None,None
4,-,The value of mindfulness and how it has arrived at Google,-,-,-,None,None
5,-,Using and abusing Python’s double-underscore methods and attributes,async/await in Python 3.5 and why it is awesome,Another pair of eyes: Reviewing code well,Hands-on with nilearn for Neuroimaging,None,None
6,Python as the keystone of building and testing C++ applications,Exploring Python Bytecode,Get in control of your workflows with Airflow,Test Driven Deployment with Ansible 2.0,Sponsored Talk,None,None


## Snippets

In [ ]:
get_room_schedule(weekly_schedule, 'A1')

In [ ]:
## schedules by room
# tabulate(get_room_schedule(weekly_schedule, 'A1'),  header=['A1'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'A2'),  header=['A2'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'A3'),  header=['A3'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'Ba1'), header=['Barria 1'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'Ba2'), header=['Barria 2'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'E'), header=[room_names[6]] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, room_names[7]]), header=[room_names[7]]] + weekday_names)

In [ ]:
def find_talk(talk_title):
    return [talk for talk in all_talks if talk_title in talk['title']]

find_talk("So, what's all the fuss about Docker?")